In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(r"D:\Ultimate Programming\Data Bases\Project Dataset\price of a Football player.csv")
df.head(2)

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
0,Alexis Sanchez,Arsenal,28,LW,1,65.0,4329,12.0,17.10%,264,3.0,Chile,0,4,1,1,0
1,Mesut Ozil,Arsenal,28,AM,1,50.0,4395,9.5,5.60%,167,2.0,Germany,0,4,1,1,0


In [3]:
df['market_value'] = df.pop('market_value')

q1 = df['page_views'].quantile(0.25)
q3 = df['page_views'].quantile(0.75)
iqr = q3 - q1
min_range = q1 - iqr*1.5
max_range = q3 + iqr*1.5

df = df[df['page_views']<max_range]

In [4]:
# Encoding the dataset
from sklearn.preprocessing import LabelEncoder
name = df.drop('name', axis=1, inplace=True)
le = LabelEncoder()
for col in df.select_dtypes(include='object').columns:
    df[col] = le.fit_transform(df[col])

In [5]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
for col in df.select_dtypes(include=['int64', 'float64']).columns :
    df[col] = ss.fit_transform(df[[col]])

In [6]:
df['region'] = df['region'].fillna(df['region'].mean())

In [7]:
df.isnull().sum()

club            0
age             0
position        0
position_cat    0
page_views      0
fpl_value       0
fpl_sel         0
fpl_points      0
region          0
nationality     0
new_foreign     0
age_cat         0
club_id         0
big_club        0
new_signing     0
market_value    0
dtype: int64

In [9]:
x = df.iloc[:, :-1]
y = df['market_value']

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [18]:
from sklearn.neighbors import KNeighborsRegressor
knr = KNeighborsRegressor(n_neighbors=6, leaf_size=2, weights='distance')
knr.fit(x_train ,y_train)

KNeighborsRegressor(leaf_size=2, n_neighbors=6, weights='distance')

In [19]:
ds = [{
    'n_neighbors' : [i for i in range(2, 20)],
    'weights' : ['uniform', 'distance'],
    'leaf_size' : [i for i in range(2, 30)],
}]

In [20]:
from sklearn.model_selection import GridSearchCV

gd = GridSearchCV(KNeighborsRegressor(), param_grid=ds)
gd.fit(x_train, y_train)

GridSearchCV(estimator=KNeighborsRegressor(),
             param_grid=[{'leaf_size': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                        14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
                                        24, 25, 26, 27, 28, 29],
                          'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 14, 15, 16, 17, 18, 19],
                          'weights': ['uniform', 'distance']}])

In [16]:
gd.best_params_

{'leaf_size': 2, 'n_neighbors': 6, 'weights': 'distance'}

In [17]:
gd.best_score_

np.float64(0.6900705779437056)

In [22]:
knr.score(x_test, y_test)*100

59.35112316322591

In [25]:
y_pred = knr.predict(x_test)

In [28]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

print(mean_squared_error(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

0.4015135370046938
0.47419092929478984
0.593511231632259


In [29]:
import joblib

In [31]:

joblib.dump(knr, 'model.joblib')  
joblib.dump(le, 'label_encoder.joblib')  
joblib.dump(ss, 'scaler.joblib') 


['scaler.joblib']